In [ ]:
import sys
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [ ]:
# Create a authentication package with Consumer key and a redirect URL
# Will ridirect to above URL if consumer key and authentication is correct
cons_key = 'YOUR_CONSUMER_KEY'  # Insert your consumer key here 
redir_url = 'http://www.shanklab.com' # Insert redirect URL here 


auth_params = {'consumer_key': cons_key, 'redirect_uri': redir_url}

token = requests.post('https://getpocket.com/v3/oauth/request', data=auth_params)

In [ ]:
# Get the generated code from the request
# The generated code is in Bytes, converted to strings to retrieve just the code 
code = str(token.content,'utf-8').split('=')[1]

In [ ]:
# Post the gen_html in the browser and authorize  
# If the authorization goes well then it should redirect to your URL
gen_html = "https://getpocket.com/auth/authorize?request_token={0}&redirect_uri={1}".format(code,redir_url)
print(gen_html)

In [ ]:
# Send the authentication package to Pocket to authorize and access the user(you) content
user_params = {'consumer_key': cons_key, 'code': code}
user = requests.post('https://getpocket.com/v3/oauth/authorize', data=user_params)

In [ ]:
# Get the access token for current session
tokens = str(user.content, 'utf-8').split('&')
access_token = tokens[0].split('=')[1]

In [ ]:
# Perpare the parameters with session token and consumer key to request Pocket app 
# Detail Type ensures the type of data we want, Complete ensures we get the Tags as well
params = {'consumer_key': cons_key, 
             'access_token':access_token,
            'detailType': 'complete'
            }

In [ ]:
# Finally send post request to get the data which are the stored posts url
result = requests.post('https://getpocket.com/v3/get', data=params)

In [ ]:
# see the resultant text we get after request
result.text

In [ ]:
# Convert the String result in JSON format to access the contents like URL and TAGS
js_format = json.loads(result.text)
jd_data = js_format['list']


list_urls = []
list_tags = []

# Storing the Tags and URLs for use later by looping through values
for data in jd_data.values():

    list_urls.append(data.get('resolved_url'))
    list_tags.append(list(data.get('tags').keys()))
    

In [ ]:
# Get the 'i' and 'ni' tags for labels and append it into list with URLs
main_list = []
for ind in range(len(list_tags)):
    if 'ni' in list_tags[ind]:
        main_list.append([list_urls[ind],'ni'])
    elif 'i' in list_tags[ind]:
        main_list.append([list_urls[ind],'i'])

In [ ]:
# Make a pandas Data Frame for later use
i_frame = pd.DataFrame(main_list, columns=['urls','interested'])

In [ ]:
# We have the URL but unfortunately not enough to train on, we'll need the whole article body
i_frame.head(10)

In [ ]:
# This function takes in a URL and gets the content of the Article
def get_content(url):
    
    
    MERC_API_KEY = 'SOME_KEY' # Insert your API key here
    merc_headers = {
    'x-api-key': MERC_API_KEY,
    }

    merc_params = (
        ('url', url),
    )

    response = requests.get('https://mercury.postlight.com/parser', headers=merc_headers, params=merc_params)
    
    jd_data = json.loads(response.text)
    
    try:
            
        data = [jd_data['title'],jd_data['content'],jd_data['domain'],jd_data['word_count']]
#         print(jd_data['title'])
    
    except KeyError:
        
        data = ['', '', '', '']
    
    return data

In [ ]:
# Get all the URLs
u_vals = i_frame['urls'].values

In [288]:
# Testing the function
get_content('')

<class 'KeyError'>


['', '', '', '']

In [ ]:
# Get the content by passing all the URL
df_u_vals = []
for i in u_vals:
    df_u_vals.append(get_content(i)) 

In [ ]:
# Store the result in Pandas DataFrame format
main_data = pd.DataFrame(df_u_vals, columns=['title','content','domain','word_count'])
main_data.head(10)

In [ ]:
# This function retrieves the raw text from HTML body 
def get_text(html):
    soup = BeautifulSoup(html, 'lxml')
    text = soup.get_text()
    return text

In [ ]:
# Map the get content text with above function and store it in a csv file for later use
# main_data.loc[:,'text'] = main_data.loc[:,'content'].map(get_text)
# main_data.to_csv("training_data.csv", index=False)

In [3]:
# Getting my already collected train data 
training_data = pd.read_csv('main_train_v2.csv')
# training_data = training_data.dropna()
training_data.head(10)

,title,content,domain,word_count,text,interested
0,Starbucks Aims for More Mobile Orders,"<div><p class=""byline""> <span>By</span> </p><d...",www.wsj.com,54.0,By Starbucks Corp. wants to spread its mobi...,ni
1,"NPR Survey: Still On Facebook, But Worried","<div><div id=""storytext"" class=""storytext stor...",www.npr.org,805.0,\n\n\n\n\n\n\nFacebook CEO Mark Zuckerberg wal...,i
2,"Mat baits, hooks and destroys pollutants in wa...","<div><p id=""first"" class=""lead"">A polymer mat ...",www.sciencedaily.com,838.0,A polymer mat developed at Rice University has...,ni
3,How often do medical problems lead to bankruptcy?,"<div><p id=""first"" class=""lead"">A team of rese...",www.sciencedaily.com,866.0,A team of researchers led by an MIT economist ...,ni
4,Brain-like computers moving closer to cracking...,"<div><p id=""first"" class=""lead"">U.S. Army Rese...",www.sciencedaily.com,994.0,U.S. Army Research Laboratory scientists have ...,i
5,Reddit bans communities under policy changes t...,"<div><section class=""article-content blueprint...",mashable.com,422.0,\nReddit is cracking down on community marketp...,i
6,Administering antibiotics through the cornea,"<div><p id=""first"" class=""lead"">Researchers fr...",www.sciencedaily.com,645.0,Researchers from Valencia's UCH CEU University...,i
7,"Don't forget: Instagram is creepy, too","<div><section class=""article-content blueprint...",mashable.com,454.0,"\nSay it with me: Instagram is bad, too. \nWit...",i
8,The strongest storm of 2018 is swirling in the...,"<div><section class=""article-content blueprint...",mashable.com,267.0,\nThe year's first Category 5 storm has formed...,ni
9,Creepy AI scans a driver's face and voice to m...,"<div><section class=""article-content blueprint...",mashable.com,254.0,"\n\n\n\nYou're being watched.\nNo, not on Face...",i


In [ ]:
# Extracting the tf-id Frequency ID Vector for our text values
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
# ngram : How is the document tokenized (1 to 3 words as a token)
# Stop words : pass all the english words to remove stop words 
# Removes words from consideration if it doesn't appear in at least three documents
vect = TfidfVectorizer(ngram_range=(1,3),stop_words='english', min_df=3)
tv = vect.fit_transform(training_data['text'])

In [223]:
tv.shape

(1075, 25163)

In [224]:
# Load are SVM model and fit the data
from sklearn.svm import LinearSVC
clf = LinearSVC()
model = clf.fit(tv, training_data['interested'])
model

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [ ]:
# Get real time data from gspeard 
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import json

In [ ]:
# Load the JSON file after creating the app with Google developer console
path_to_json_file = "PATH_TO_JSON_FILE" # insert the path to json file
json_key = json.load(open(path_to_json_file))
# Retrieve the client email and use that to give permission to access the Google Sheet 
# json_key['client_email']

In [ ]:
# Make sure to enable the Sheets and Google Drive API from Google Developer Console
# Construct the Package to send for authorization 
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(path_to_json_file, scope)
gc = gspread.authorize(credentials)

In [ ]:
# After giving access to the sheet by sharing it with Client Email retreived from above
# Open the Sheet by giving it Sheet ID retrieved from the Sheets URL 
sheet_id = "SHEET_ID_SHOWN_IN_URL" # Enter your sheet id
real_stories = gc.open_by_key(sheet_id)

In [297]:
# Load the Sheet values
sheet = real_stories.sheet1
real_data = list(zip(sheet.col_values(1),sheet.col_values(2),sheet.col_values(3),sheet.col_values(4), sheet.col_values(5)))

# Store the retrieved data into data frame 
real_time_df = pd.DataFrame(real_data, columns=['DateTime','Title','URL','Body','Image URL'])
real_time_df.replace('',pd.np.nan, inplace=True)
real_time_df = real_time_df.dropna()
real_time_df.head(10)

,DateTime,Title,URL,Body,Image URL
0,"March 31, 2018 at 11:00AM",Google needs your help finding Waldo,https://techcrunch.com/2018/03/31/google-needs...,"<p>At some point in the not-so-distant past, A...",https://techcrunch.com/wp-content/uploads/2018...
1,"March 31, 2018 at 11:12AM",Get ready for Hasbro's biggest Star Wars toy y...,https://www.cnet.com/news/largest-star-wars-to...,Hasbro's new HasLab campaign wants to make a h...,http://ifttt.com/images/no_image_card.png
2,"March 31, 2018 at 11:19AM",Westworld' season 2 on HBO: Everything we know...,https://www.cnet.com/news/westworld-season-two...,Not only is the disturbing Old West theme park...,http://ifttt.com/images/no_image_card.png
3,"March 31, 2018 at 11:30AM",Exploring a controversial net neutrality opini...,http://feedproxy.google.com/~r/TheNextWeb/~3/o...,"<img src=""https://cdn0.tnwcdn.com/wp-content/b...",https://cdn0.tnwcdn.com/wp-content/blogs.dir/1...
4,"March 31, 2018 at 11:30AM",Red Hat looks beyond Linux,https://techcrunch.com/2018/03/31/red-hat-look...,"<p>The <a href=""https://www.redhat.com/en"">Red...",https://techcrunch.com/wp-content/uploads/2018...
5,"March 31, 2018 at 12:00PM","The best, worst, and weirdest cars at the 2018...",https://www.theverge.com/2018/3/31/17182178/ny...,"<img alt="""" data-portal-copyright=""Photo By Am...",https://cdn.vox-cdn.com/thumbor/nqpkGGVQ5Q2zrp...
6,"March 31, 2018 at 12:00PM",Who gains from Facebook’s missteps?,https://techcrunch.com/2018/03/31/who-gains-fr...,"<p>When <a class=""crunchbase-link"" href=""https...",http://ifttt.com/images/no_image_card.png
7,"March 31, 2018 at 12:49PM",Knitting machines power up with computer-gener...,https://techcrunch.com/2018/03/31/knitting-mac...,"<p>At last, a use for that industrial knitting...",https://techcrunch.com/wp-content/uploads/2018...
8,"March 31, 2018 at 12:56PM","Trump again attacks Amazon, alleging 'Post Off...",https://www.cnet.com/news/trump-again-attacks-...,The president tweets against Amazon's use of t...,http://ifttt.com/images/no_image_card.png
9,"March 31, 2018 at 01:00PM",How an extra lens can transform your iPhone’s ...,https://www.theverge.com/circuitbreaker/2018/3...,"<img alt="""" data-portal-copyright="""" data-has-...",https://cdn.vox-cdn.com/thumbor/XuE-yeu8xsmQp_...


In [ ]:
# sheet.clear()

In [ ]:
# Retrieve the body content from urls
real_content = []
real_ind = 0
for url in real_time_df['URL'].values:
    content = get_content(url)
    
    if content[0] == 'Forbes Welcome':
        content[0],content[1] = real_time_df.iloc[real_ind]['Title'], real_time_df.iloc[real_ind]['Body']
    
    if content[0] == '':
        content = [real_time_df.iloc[real_ind]['Title'], real_time_df.iloc[real_ind]['Body'], real_time_df.iloc[real_ind]['URL'],1]
    
    print(real_ind,content[0])
    real_content.append(content)
    real_ind += 1

In [290]:
ret_data = pd.DataFrame(real_content, columns=['title','content','domain','word_count'])
# ret_data.dropna(inplace=True)
ret_data.loc[:,'text'] = ret_data.loc[:,'content'].map(get_text)
ret_data.head(10)

,title,content,domain,word_count,text
0,Google needs your help finding Waldo,"<div class=""article-content""> <p>At some point...",techcrunch.com,216,"At some point in the not-so-distant past, Apr..."
1,"Hasbro's biggest Star Wars toy yet, Jabba's ba...","<div><div class=""col-7 article-main-body row"">...",www.cnet.com,372,\n\n\n\n\nStar Wars fans may want to make room...
2,'Westworld' is back: Everything we know about ...,"<div><div class=""col-7 article-main-body row"">...",www.cnet.com,1165,"\nReady to go back to ""Westworld""? Just like ""..."
3,Exploring a controversial net neutrality opini...,"<article class=""post""> <div class=""post-imageB...",thenextweb.com,1,Washington recently became the nation’s...
4,Red Hat looks beyond Linux,"<div class=""article-content""> <p>The <a href=""...",techcrunch.com,964,The Red Hat Linux distribution is turning 25 ...
5,"The best, worst, and weirdest cars at the 2018...","<div><div class=""c-entry-content"">\n <p id=""o...",www.theverge.com,1058,\nThe New York International Auto Show opens t...
6,Who gains from Facebook’s missteps?,"<div class=""article-content""> <p>When <a class...",techcrunch.com,964,"When Facebook loses, who wins?\nThat’s a que..."
7,Knitting machines power up with computer-gener...,"<div class=""article-content""> <p>At last, a us...",techcrunch.com,243,"At last, a use for that industrial knitting m..."
8,"Trump again attacks Amazon, alleging 'Post Off...","<div><div class=""col-7 article-main-body row"">...",www.cnet.com,762,"\nAmazon and its CEO, Jeff Bezos, have been fr..."
9,How an extra lens can transform your iPhone’s ...,"<div><div class=""c-entry-content"">\n <p id=""n...",www.theverge.com,367,"\nA few years ago, the hit Sundance film Tange..."


In [263]:
# Transform the text to already trained Vectorized model of trained data
pred_vect = vect.transform(ret_data['text'])
print(pred_vect.shape)

# Use our trained value to predict the data 
new_pred = clf.predict(pred_vect)

(200, 25163)


In [289]:
new_pred = pd.DataFrame(new_pred,columns=['interested'])
new_pred.head()

,interested
0,i
1,ni
2,ni
3,i
4,i


In [180]:
# Get the Text value from the Body, if you wante try predicting values using body given in google sheets
real_time_df.loc[:,'text'] = real_time_df.loc[:,'Body'].map(get_text)
real_time_df.shape

(484, 6)

In [291]:
pred_all_data = pd.merge(ret_data,new_pred, left_index=True,right_index=True)
pred_all_data.head(10)

,title,content,domain,word_count,text,interested
0,Google needs your help finding Waldo,"<div class=""article-content""> <p>At some point...",techcrunch.com,216,"At some point in the not-so-distant past, Apr...",i
1,"Hasbro's biggest Star Wars toy yet, Jabba's ba...","<div><div class=""col-7 article-main-body row"">...",www.cnet.com,372,\n\n\n\n\nStar Wars fans may want to make room...,ni
2,'Westworld' is back: Everything we know about ...,"<div><div class=""col-7 article-main-body row"">...",www.cnet.com,1165,"\nReady to go back to ""Westworld""? Just like ""...",ni
3,Exploring a controversial net neutrality opini...,"<article class=""post""> <div class=""post-imageB...",thenextweb.com,1,Washington recently became the nation’s...,i
4,Red Hat looks beyond Linux,"<div class=""article-content""> <p>The <a href=""...",techcrunch.com,964,The Red Hat Linux distribution is turning 25 ...,i
5,"The best, worst, and weirdest cars at the 2018...","<div><div class=""c-entry-content"">\n <p id=""o...",www.theverge.com,1058,\nThe New York International Auto Show opens t...,ni
6,Who gains from Facebook’s missteps?,"<div class=""article-content""> <p>When <a class...",techcrunch.com,964,"When Facebook loses, who wins?\nThat’s a que...",i
7,Knitting machines power up with computer-gener...,"<div class=""article-content""> <p>At last, a us...",techcrunch.com,243,"At last, a use for that industrial knitting m...",i
8,"Trump again attacks Amazon, alleging 'Post Off...","<div><div class=""col-7 article-main-body row"">...",www.cnet.com,762,"\nAmazon and its CEO, Jeff Bezos, have been fr...",ni
9,How an extra lens can transform your iPhone’s ...,"<div><div class=""c-entry-content"">\n <p id=""n...",www.theverge.com,367,"\nA few years ago, the hit Sundance film Tange...",ni


In [ ]:
def show_accuracy(actual_vals,pred_val):
    
    accuracy_lst = []
    for i in range(len(actual_vals)):
        if actual_vals[i] == pred_val[i]:
            accuracy_lst.append(0)
        else:
            accuracy_lst.append(1)

    total_items = len(actual_vals)
    zeros = total_items - np.count_nonzero(accuracy_lst)
    acc_percent = ( zeros / total_items ) * 100
    print("Accuracy : ", acc_percent)

In [172]:
# Accuracy was calculated using previous trained model
actual_vals = real_time_df['Interest'].values
show_accuracy(actual_vals,pred_val)

Accuracy :  63.13725490196078


In [228]:
test_data = pd.read_csv('main_test.csv')
test_data = test_data.dropna()
test_tv = vect.transform(test_data['text'])
print(test_tv.shape)
test_pred = clf.predict(test_tv)
show_accuracy(test_data['interested'].values,test_pred)

(591, 25163)
Accuracy :  98.30795262267344
